In [1]:
%load_ext autoreload
%autoreload 2

# North Sea Wind Farms

## Goal

The goal of this notebook is to collect (spatial) data and generate statistics on wind farms in the North Sea. We want information on:
1. Spatial status and changes
2. Temporal changes
3. Technical specifics on turbines (power, infrastructure incl. cabels, materials, supply chain)
5. Ownership
6. End-users

## Data sources

### Location and ownership

1. MapStand: installed and planned windfarms (wfs)
2. Global Offshore Wind Farm Database And Intelligence, 4COffshore ([source](https://www.4coffshore.com/windfarms/)). Holds information on location, ownership and production of wind farms. Is a commercial provider. Some data seem a bit messy (e.g. the categorisation into sea areas is not consistent. 
3. Wind Europe Public database ([source](https://windeurope.org/intelligence-platform/product/european-offshore-wind-farms-map-public/)). Holds information on location and ownership. Is a lobby organisation. Dataset is probably not complete. 
4. EMODnet Wind Turbines ([source](https://emodnet.ec.europa.eu/geonetwork/emodnet/eng/catalog.search#/search?any=EMODnet%20Human%20Activities,%20Wind%20Farms)). Norway data is lacking. Data is current and has been extensively verified up until January 1st this year, so this is a very useful resource.
5. Crown Estate offshore wind farm onwership tables ([source](https://www.thecrownestate.co.uk/en-gb/what-we-do/on-the-seabed/energy/offshore-wind-farm-ownership/#OFTOownership)). More data available at the Crown Estate open data portal ([source](https://opendata-thecrownestate.opendata.arcgis.com/search?groupIds=f0d0ec92da76434d9e91f2e4dcb3a99f)). And here for Scotland ([source](https://crown-estate-scotland-spatial-hub-coregis.hub.arcgis.com/search?tags=offshore%20wind)). Official UK registries. Data quality is presumed to be good.
6. NVE data from Norway ([source](https://nedlasting.nve.no/gis/)). Offical Norwegian registry. Data quality is presumed to be good.
7. Netherlands wind turbines from RIVM ([source](https://data.rivm.nl/meta/srv/dut/catalog.search#/search?resultType=details&sortBy=relevance&fast=index&_content_type=json&from=1&to=20&any=Windturbines%20-%20vermogen)). Data quality is good. Is point data instead of polygons. Info on individual turbines (height, power, location).
8. Denmark
9. Germany
10. Belgium

### Tenders and supply chain
1. Bloomberg Terminal (at office of FTM): contains information on the supply chain of wind turbine manufacturers. Some data might be outdated. 
2. Aleph (FTM data repository) contains all European Tenders

In [12]:
import pandas as pd
import geopandas as gpd
from requests import Request
import ast
import yaml
from owslib.wfs import WebFeatureService 
from dotenv import load_dotenv
import os

pd.options.display.max_columns = 50
load_dotenv('../../.env')

True

In [5]:
# Set path

PATH_DATA = '../../drive/energy/renewables/'

# Get API_KEY

API_KEY = os.environ.get('MAPSTAND_API_KEY')

# Import config

with open('../../config/ontology.yaml') as yaml_file:
    config = yaml.safe_load(yaml_file)

cols_clean = config.get('infra_columns').get('wind')
values_clean = config.get('infra_values').get('wind')

#### MapStand

The data from MapStand is probably sufficient.

In [9]:
url = f'https://hub.mapstand.com/gs/ows?VERSION=1.3.0&apikey={API_KEY}'
wfs = WebFeatureService(url)

In [19]:
layers = ['mps:powerplant_windfarm_planned', 'mps:powerplant_windfarm_installed']
dfs = []

for layer in layers:
    
    params = dict(service='WFS', version="1.3.0", request='GetFeature',
          typeName=layer, outputFormat='json')
    
    wfs_request_url = Request('GET', url, params=params).prepare().url
    
    df = gpd.read_file(wfs_request_url)
    df['dataset'] = layer.replace('mps:powerplant_windfarm_', '')
    dfs.append(df)

df = pd.concat(dfs)

In [31]:
df.to_csv('../../drive/energy/renewables/mapstand_windfarms.csv', index=False)

In [22]:
# clean dataset

countries = ['Netherlands', 'United Kingdom', 'Belgium', 'Norway', 'Denmark', 'Germany']

ns = df[(df.admin_area_name.isin(countries)) & (df.mps_est_shore_status == 'OFFSHORE')].copy()
len(ns)

408

In [29]:
ns.columns

Index(['id', 'admin_area_name', 'attribution', 'capacity_mw', 'coast_distance',
       'description', 'installation_year', 'mps_created_time',
       'mps_datasource_tags', 'mps_est_area_sqkm', 'mps_est_coast_distance_km',
       'mps_est_elevation_max_m', 'mps_est_elevation_min_m',
       'mps_est_shore_status', 'mps_project_tag', 'mps_uuid', 'name',
       'number_generators', 'operator', 'operator_group', 'owner',
       'owner_group', 'round_name', 'status', 'updt', 'windfarm_type',
       'geometry', 'dataset', 'closure_year', 'decommissioning_year'],
      dtype='object')

In [30]:
ns.groupby('admin_area_name').capacity_mw.sum()

admin_area_name
Belgium             2320.00
Denmark            20628.44
Germany            59564.22
Netherlands         7404.40
Norway             14547.80
United Kingdom    101298.90
Name: capacity_mw, dtype: float64

#### 4COffshore

The latitude is given, but the longitude is not. We therefore might need to link the data on project names

In [4]:
# Import 4COffshore

fourc = pd.read_csv(f'{PATH_DATA}4c_offshore_freemium_windfarms.csv')

# Filter out North Sea Countries

fourc = fourc[fourc.country_filter.isin(['netherlands', 'norway', 'united-kingdom', 'germany', 'denmark', 'belgium'])].copy()

# Rename columns

fourc = fourc.rename(columns=cols_clean)

# Clean latitude column

fourc.latitude = fourc.latitude.str.replace('°', '', regex=False)

# Clean power_mw column

fourc.power_mw = fourc.power_mw.str.replace(' MW', '').astype(float)
            
# Select relevant columns

fourc = fourc[['name', 'other_names', 'country', 'owner', 'developers', 'status', 'power_mw', 'category_round', 'main_url', 'comments', 'turbine_model', 'foundation_type', 'latitude']].copy()

# Clean owners to proper list

fourc[['developers', 'owner']] = fourc[['developers', 'owner']].applymap(lambda x: ast.literal_eval(x))

# Get list of project names for comparison

fourc_names = list(set(fourc.name.str.lower()))
len(fourc)

772

#### Wind Europe

The dataset contains coordinates, so we can perform a spatial join to link the data to the polygons of EMODNet. But let's also get the project names to see if we can match some of the data from 4COffshore. There might be some wind farms outside of the North Sea, but we can use a spatial mask to filter these out.

In [5]:
# Import Wind Europe

weurope = pd.read_csv(f'{PATH_DATA}windeurope.csv', sep=';')

# Clean columns

weurope.columns = weurope.columns.str.lower().str.replace(' ', '_', regex=False)

# Drop some columns

weurope = weurope.drop([col for col in weurope.columns if '(group)' in col], axis=1)

# Rename columns

weurope = weurope.rename(columns=cols_clean)

# Select relevant columns

weurope = weurope[['name', 'status', 'power_mw', 'year', 'year_awarded', 'foundation_type', 'owner', 'turbine_manufacturer', 'latitude', 'longitude']].copy()

# Get list of project names for comparison

weurope_names = list(set(weurope['name'].str.lower()))
len(weurope_names)

138

#### EMODnet

This data is the basis of our efforts, because it's very current and reliable. 

In [6]:
# Import EMODnet

emod = gpd.read_file(f'{PATH_DATA}EMODnet_HA_WindFarms_20221219/EMODnet_HA_WindFarms_pg_20221219.shp')# Filter out North Sea countries

# Filter North Sea countries

emod = emod[emod.COUNTRY.isin(['Denmark', 'United Kingdom', 'Netherlands', 'Germany', 'Belgium'])].copy()

# Column names to lowercase

emod.columns = emod.columns.str.lower()

# Select relevant columns

emod = emod[['country', 'name', 'n_turbines', 'power_mw', 'status', 'year', 'geometry']].copy().reset_index().drop('index', axis=1)

# Get list of project names for comparison

emod_names = list(set(emod.name.str.lower()))
len(emod_names)

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


322

In [9]:
emod.to_csv(PATH_DATA + 'emod.csv', index=True, sep=',')

#### NVE Norwegian data

There are two slightly different datasets available:
1. From the NVE map interface (norwegian_wind_farms.geojson). This is the most elaborate one and also contains ownership information. 
2. Also from the NVE but from the open data service. This has fewer data fields, but contain 4 more records.

In [7]:
nve = gpd.read_file(f'{PATH_DATA}norwegian_wind_farms.geojson')

# Clean columns and values

nve = nve.rename(columns=cols_clean)
nve = nve[nve.name.str.contains('offshore')==True]
nve.status = nve.status.replace(values_clean, regex=False)
nve.phase = nve.phase.replace(values_clean, regex=False)

# Select relevant columns

nve = nve[[col for col in nve.columns if col.islower()]].drop(['datafangstdato', 'malemetode', 'noyaktighet'], axis=1).copy()
len(nve)

32

In [10]:
nve.to_csv(PATH_DATA + 'nve.csv', index=True, sep=',')

In [12]:
fourc.to_csv(PATH_DATA + '4coffshore.csv', sep=',')
weurope.to_csv(PATH_DATA + 'windeurope.csv', sep=',')

In [12]:
country = 'United Kingdom'
fourc[fourc.country==country][['name', 'other_names', 'latitude']].sort_values(by='name')[40:55]

,name,other_names,latitude
1486,Central North Sea Electrification,NaN,57.111656
1325,Cirrus Shell Flat Array,"Cirrus Array, Shell flats",53.892707
1344,Cromer,NaN,53.005326
1509,Culzean Floating Wind Pilot Project,TotalEnergies INTOG,57.197453
1291,Docking Shoal,NaN,53.15441
1310,Dogger Bank - Teesside C,"Zone 3, Todd Point A (Tranche C)",55.245447
1311,Dogger Bank - Teesside D,"Zone 3, Todd Point B (Tranche C)",55.30495
1497,Dogger Bank A,"Creyke Beck A, Zone 3, Project One",54.769475
1296,Dogger Bank B,"Creyke Beck B, Zone 3",54.977377
1308,Dogger Bank C,"Teesside A, Zone 3, Lackenby A (Tranche B)",55.031196


In [13]:
emod[emod.country==country][['name', 'geometry']].sort_values(by='name')[25:40]

,name,geometry
217,Connel (PLAT-I),"POLYGON ((-5.39916 56.45586, -5.39877 56.45624..."
174,Creyke Beck A Wind Farm,"POLYGON ((1.97695 54.65929, 1.63288 54.74169, ..."
175,Creyke Beck B Wind Farm,"POLYGON ((1.85498 55.10215, 1.86187 54.85924, ..."
22,Docking Shoal Met Mast,"POLYGON ((0.64789 53.15873, 0.64802 53.15872, ..."
172,Dogger Bank - Teesside A,"POLYGON ((3.03444 54.96115, 3.03187 54.95485, ..."
285,Dogger Bank A,"POLYGON ((1.97695 54.65929, 1.63288 54.74168, ..."
286,Dogger Bank B,"POLYGON ((1.85498 55.10215, 1.86187 54.85924, ..."
292,Dogger Bank C,"POLYGON ((3.03444 54.96115, 3.03187 54.95484, ..."
331,Dogger Bank East Met Mast,"POLYGON ((2.70276 55.10165, 2.70297 55.10164, ..."
332,Dogger Bank West Met Mast,"POLYGON ((1.82018 54.86919, 1.82039 54.86918, ..."
